# Claude API를 활용한 비디오/이미지 분석

이 노트북은 추출된 이미지와 동영상을 Claude API(Claude 3.5 Sonnet 등)를 사용하여 분석하는 예제입니다.

In [ ]:
# 필요한 라이브러리 설치
%pip install anthropic opencv-python joblib scikit-learn

In [ ]:
import os
import base64
import cv2
from anthropic import Anthropic

# ========== 설정 ==========
# Claude API Key 설정
# (사용자가 입력한 키 유지)
API_KEY = ''

# 분석할 이미지들이 저장된 폴더 경로 (이미지 분석용)
IMAGE_DIR = r"sample\output\군집\E04_001_images"

# 분석할 동영상 파일 경로 (동영상 분석용)
VIDEO_PATH = r"sample\output\침입\E01_001.mp4"

# 사용할 모델
MODEL_NAME = "claude-sonnet-4-5" 
# 참고: 실제 API 모델명은 "claude-3-5-sonnet-20241022" 입니다. 오류 발생 시 변경하세요.

# 클라이언트 초기화
client = Anthropic(api_key=API_KEY)
# os.environ.get("ANTHROPIC_API_KEY") 를 사용하려면 위 줄 대신 아래 주석 해제
# client = Anthropic()

In [23]:
def encode_image_from_file(image_path):
    """이미지 파일을 base64로 인코딩합니다."""
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def encode_frame(frame, quality=70):
    """OpenCV 프레임(numpy array)을 base64로 인코딩합니다. (품질 조절 가능)"""
    # JPEG 품질 설정 (0~100, 기본 95). 413 에러 방지를 위해 낮춤.
    encode_param = [int(cv2.IMWRITE_JPEG_QUALITY), quality]
    _, buffer = cv2.imencode('.jpg', frame, encode_param)
    return base64.b64encode(buffer).decode('utf-8')

def call_claude_api(messages_content):
    """Claude API를 호출하는 내부 함수"""
    print("\n[요청] Claude API 분석 요청 중...")
    message = client.messages.create(
        model=MODEL_NAME,
        max_tokens=1024,
        messages=[
            {
                "role": "user",
                "content": messages_content
            }
        ]
    )
    return message.content[0].text

def analyze_images(image_paths, prompt):
    """
    여러 장의 이미지를 Claude에게 전송하여 분석을 요청합니다.
    """
    messages_content = []
    
    for path in image_paths:
        if not os.path.exists(path):
            print(f"[경고] 파일을 찾을 수 없음: {path}")
            continue
            
        base64_image = encode_image_from_file(path)
        messages_content.append({
            "type": "image",
            "source": {
                "type": "base64",
                "media_type": "image/jpeg",
                "data": base64_image
            }
        })
        print(f"[준비] 이미지 추가됨: {os.path.basename(path)}")
    
    messages_content.append({"type": "text", "text": prompt})
    return call_claude_api(messages_content)

def analyze_video(video_path, prompt, interval_seconds=1, max_frames=20):
    """
    동영상 파일을 직접 읽어서 프레임을 추출하고 분석합니다.
    
    Args:
        interval_seconds: 몇 초마다 프레임을 추출할지 (기본 1초)
        max_frames: 최대 분석할 프레임 수 (API 비용 절감 및 토큰 제한 방지)
    """
    if not os.path.exists(video_path):
        return f"[오류] 비디오 파일을 찾을 수 없습니다: {video_path}"
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return f"[오류] 비디오를 열 수 없습니다."

    fps = cap.get(cv2.CAP_PROP_FPS)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    
    print(f"[정보] 비디오 로드: {os.path.basename(video_path)} (FPS: {fps:.2f}, 총 {total_frames} 프레임)")
    
    frame_interval = int(fps * interval_seconds)
    if frame_interval == 0: frame_interval = 1
    
    messages_content = []
    frame_count = 0
    extracted_count = 0
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # 지정된 간격마다 프레임 선택
        if frame_count % frame_interval == 0:
            # 프레임 리사이즈 및 압축 (413 Request Too Large 에러 방지)
            h, w = frame.shape[:2]
            
            # 1. 해상도 축소 (1024px -> 512px)
            target_w = 1024
            if w > target_w:
                new_w = target_w
                new_h = int(h * (target_w / w))
                frame = cv2.resize(frame, (new_w, new_h))
            
            # 2. JPEG 품질 60%로 압축 (용량 감소)
            base64_image = encode_frame(frame, quality=60)
            
            messages_content.append({
                "type": "image",
                "source": {
                    "type": "base64",
                    "media_type": "image/jpeg",
                    "data": base64_image
                }
            })
            extracted_count += 1
            print(f"[준비] 프레임 {frame_count} 추가됨 (총 {extracted_count}장)")
            
            if extracted_count >= max_frames:
                print(f"[정보] 최대 프레임 수({max_frames})에 도달하여 추출을 중단합니다.")
                break
        
        frame_count += 1
        
    cap.release()
    
    if extracted_count == 0:
        return "[오류] 추출된 프레임이 없습니다."
        
    messages_content.append({"type": "text", "text": prompt})
    return call_claude_api(messages_content)

In [29]:
# ========== 실행 예제 (2): 동영상 직접 분석 (정밀 모드) ==========
VIDEO_PATH = r"sample\output\침입\E01_001.mp4"

print(f"\n[동영상 분석] {VIDEO_PATH} 분석 시작...")

# 정밀 분석 프롬프트 (동용상용)
VIDEO_PROMPT = """
당신은 CCTV 분석관입니다. 이미지 속 상황을 한 문장으로 묘사하세요. (~함 체 사용, 배경 설명 금지)
"""

try:
    if os.path.exists(VIDEO_PATH):
        # 0.5초 간격으로 추출 (최대 100장까지 분석)
        # 413 에러가 해결되지 않으면 max_frames를 50으로 줄이세요.
        result_video = analyze_video(
            VIDEO_PATH, 
            VIDEO_PROMPT, 
            interval_seconds=0.5,  # 0.5초 단위로 정밀 분석
            max_frames=100         # 최대 100프레임 (512px 축소됨) 분석
        )
        print("\n========== Claude 분석 결과 (동영상 정밀 분석) ==========")
        print(result_video)
    else:
        print(f"[오류] 동영상 파일이 없습니다: {VIDEO_PATH}")
except Exception as e:
    print(f"\n[오류] 동영상 분석 중 에러 발생: {e}")


[동영상 분석] sample\output\침입\E01_001.mp4 분석 시작...
[정보] 비디오 로드: E01_001.mp4 (FPS: 30.00, 총 1029 프레임)
[준비] 프레임 0 추가됨 (총 1장)
[준비] 프레임 15 추가됨 (총 2장)
[준비] 프레임 30 추가됨 (총 3장)
[준비] 프레임 45 추가됨 (총 4장)
[준비] 프레임 60 추가됨 (총 5장)
[준비] 프레임 75 추가됨 (총 6장)
[준비] 프레임 90 추가됨 (총 7장)
[준비] 프레임 105 추가됨 (총 8장)
[준비] 프레임 120 추가됨 (총 9장)
[준비] 프레임 135 추가됨 (총 10장)
[준비] 프레임 150 추가됨 (총 11장)
[준비] 프레임 165 추가됨 (총 12장)
[준비] 프레임 180 추가됨 (총 13장)
[준비] 프레임 195 추가됨 (총 14장)
[준비] 프레임 210 추가됨 (총 15장)
[준비] 프레임 225 추가됨 (총 16장)
[준비] 프레임 240 추가됨 (총 17장)
[준비] 프레임 255 추가됨 (총 18장)
[준비] 프레임 270 추가됨 (총 19장)
[준비] 프레임 285 추가됨 (총 20장)
[준비] 프레임 300 추가됨 (총 21장)
[준비] 프레임 315 추가됨 (총 22장)
[준비] 프레임 330 추가됨 (총 23장)
[준비] 프레임 345 추가됨 (총 24장)
[준비] 프레임 360 추가됨 (총 25장)
[준비] 프레임 375 추가됨 (총 26장)
[준비] 프레임 390 추가됨 (총 27장)
[준비] 프레임 405 추가됨 (총 28장)
[준비] 프레임 420 추가됨 (총 29장)
[준비] 프레임 435 추가됨 (총 30장)
[준비] 프레임 450 추가됨 (총 31장)
[준비] 프레임 465 추가됨 (총 32장)
[준비] 프레임 480 추가됨 (총 33장)
[준비] 프레임 495 추가됨 (총 34장)
[준비] 프레임 510 추가됨 (총 35장)
[준비] 프레임 525 추가됨 (총 36장)
[준비] 프레임 540 추가됨 (총

In [ ]:
# # ========== 실행 예제 (1): 이미지 폴더 분석 ==========
# 1 프레임으로 사진 분석

# # 정밀 분석을 위한 프롬프트 수정
# PROMPT = """
# 사진에 아이들이 뭐하는지 설명해줘
# """

# # 1. 이미지 파일 목록 가져오기
# if os.path.exists(IMAGE_DIR):
#     all_images = sorted([os.path.join(IMAGE_DIR, f) for f in os.listdir(IMAGE_DIR) if f.lower().endswith('.jpg')])
    
#     # 전체 이미지 중 10장 간격으로 선택 (샘플링)
#     step = 2
#     selected_images = all_images[::step]
    
#     print(f"[이미지 분석] 총 {len(selected_images)}장 (전체 {len(all_images)}장 중 {step}장 간격) 분석 시작...")
    
#     try:
#         result = analyze_images(selected_images, PROMPT)
#         print("\n========== Claude 분석 결과 (이미지) ==========")
#         print(result)
#     except Exception as e:
#         print(f"[오류] {e}")
# else:
#     print(f"[오류] 이미지 폴더가 없습니다: {IMAGE_DIR}")